In [1]:
# Ejercicio Chat Bot

In [4]:
!pip install -q streamlit langchain langchain-openai langchain-community pypdf faiss-cpu openai tiktoken

In [3]:
%%writefile "app.py"
# Importamos las librerías necesarias para la aplicación
import streamlit as st  # Para crear la interfaz web de manera sencilla
import os  # Para interactuar con el sistema operativo (rutas de archivos, variables de entorno)

# --- IMPORTACIONES ESENCIALES Y ROBUSTAS ---
# Estas son las herramientas específicas de LangChain que usaremos
from langchain_community.document_loaders import PyPDFLoader  # Para leer el contenido de archivos PDF
from langchain_text_splitters import RecursiveCharacterTextSplitter  # Para dividir textos largos en fragmentos más pequeños y manejables
from langchain_openai import OpenAIEmbeddings, ChatOpenAI  # Para usar los modelos de OpenAI (embeddings para vectores y ChatOpenAI para el bot)
from langchain_community.vectorstores import FAISS  # Base de datos vectorial eficiente para buscar información similar
from langchain_core.prompts import ChatPromptTemplate  # Para crear plantillas de prompts estructurados para el chat
from langchain_core.messages import AIMessage, HumanMessage  # Para representar los mensajes del bot (AI) y del usuario (Human) en el historial

# Configuramos la página de Streamlit 
st.set_page_config(page_icon="🥈", page_title="Asistente Silver Economy")
st.title("🥈 Asistente Silver Economy (Funcional)")

# --- CONFIGURACIÓN DE LA API KEY DE OPENAI ---
# Verificamos si la clave ya está en las variables de entorno del sistema
if "OPENAI_API_KEY" in os.environ:
    openai_api_key = os.getenv("OPENAI_API_KEY")
else:
    # Si no está, creamos un campo de entrada de contraseña en la barra lateral para que el usuario la ingrese
    openai_api_key = st.sidebar.text_input("OpenAI API Key:", type="password")
    # Si el usuario aún no ha ingresado la clave, mostramos una advertencia y detenemos la ejecución
    if not openai_api_key:
        st.warning("Introduce tu API Key para continuar.")
        st.stop()
    # Una vez ingresada, la guardamos en las variables de entorno para que LangChain la use automáticamente
    os.environ["OPENAI_API_KEY"] = openai_api_key

# --- FUNCIÓN PARA CARGAR Y PROCESAR LOS DATOS ---
# Usamos @st.cache_resource para que esta función pesada solo se ejecute una vez y se guarde en memoria
@st.cache_resource
def iniciar_base_datos():
    # Definimos la ruta donde están los archivos PDF. "./Data" significa la carpeta 'Data' en el directorio actual.
    ruta_data = "./Data"
    # Verificamos que la carpeta exista para evitar errores
    if not os.path.exists(ruta_data):
        st.error("No se encuentra la carpeta 'Data'. Asegúrate de ejecutar esto desde la carpeta 'Tarea 4'.")
        st.stop()
    
    docs = []  # Lista para guardar todos los documentos cargados
    # Mostramos un spinner mientras se cargan los archivos para dar feedback al usuario
    with st.spinner("Cargando documentos..."):
        # Recorremos todos los archivos en la carpeta 'Data'
        for archivo in os.listdir(ruta_data):
            # Si el archivo es un PDF, lo cargamos
            if archivo.endswith(".pdf"):
                # Creamos la ruta completa al archivo
                ruta_completa = os.path.join(ruta_data, archivo)
                # Usamos PyPDFLoader para leer el PDF
                loader = PyPDFLoader(ruta_completa)
                # Añadimos el contenido del PDF a nuestra lista de documentos
                docs.extend(loader.load())
    
    # Dividimos los documentos en fragmentos (chunks) más pequeños
    # chunk_size=1000: cada fragmento tendrá aprox. 1000 caracteres
    # chunk_overlap=200: los fragmentos se solaparán en 200 caracteres para no perder contexto entre cortes
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(docs)
    
    # Creamos la base de datos vectorial (FAISS) usando los embeddings de OpenAI
    # Esto convierte los fragmentos de texto en vectores numéricos para poder buscar por similitud semántica
    return FAISS.from_documents(splits, OpenAIEmbeddings())

# --- INICIALIZACIÓN DEL ESTADO DE LA APLICACIÓN ---
# Verificamos si la base de datos vectorial ya está cargada en la sesión
if "vectorstore" not in st.session_state:
    # Si no está, llamamos a la función para cargarla
    st.session_state.vectorstore = iniciar_base_datos()
    # Creamos un 'retriever' (buscador) a partir de la base vectorial
    # search_kwargs={"k": 3} significa que buscará los 3 fragmentos más relevantes para cada pregunta
    st.session_state.retriever = st.session_state.vectorstore.as_retriever(search_kwargs={"k": 3})
    st.success("¡Sistema listo! Haz tu pregunta.")

# --- INICIALIZACIÓN DEL HISTORIAL DEL CHAT ---
# Si no existe el historial en la sesión, creamos una lista vacía para guardarlo
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

# --- CONFIGURACIÓN DEL MOTOR DEL CHAT (MODELO DE LENGUAJE) ---
# Usamos el modelo GPT-3.5 Turbo con temperatura 0 para respuestas más precisas y menos creativas (ideal para RAG)
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# --- DEFINICIÓN DE LA PLANTILLA DEL PROMPT ---
# Esta plantilla le dice al modelo cómo debe comportarse y qué formato usar
template = """Eres un asistente útil para personas mayores. Basate SOLO en el siguiente contexto para responder.
Si no lo sabes, di que no tienes esa información en tus documentos.

Contexto:
{context}

Historial:
{chat_history}

Pregunta: {question}
Respuesta:"""

# Creamos el objeto PromptTemplate a partir del texto de la plantilla
prompt_template = ChatPromptTemplate.from_template(template)

# --- FUNCIÓN RAG MANUAL (Retrieval-Augmented Generation) ---
# Esta función orquesta todo el proceso de respuesta
def responder_pregunta(pregunta, historial):
    # 1. RECUPERACIÓN (Retrieval): Buscamos los documentos más relevantes para la pregunta en nuestra base vectorial
    docs_relevantes = st.session_state.retriever.invoke(pregunta)
    # Unimos el contenido de los documentos recuperados en un solo texto
    contexto_texto = "\n\n".join([d.page_content for d in docs_relevantes])
    
    # 2. FORMATEO DEL HISTORIAL: Convertimos la lista de mensajes en un texto simple para el prompt
    # Tomamos solo los últimos 4 mensajes para no sobrecargar el contexto
    historial_texto = "\n".join([f"{msg.type}: {msg.content}" for msg in historial[-4:]])
    
    # 3. GENERACIÓN DEL PROMPT: Rellenamos la plantilla con el contexto, historial y la pregunta actual
    prompt_final = prompt_template.format_messages(
        context=contexto_texto,
        chat_history=historial_texto,
        question=pregunta
    )
    
    # 4. LLAMADA AL MODELO: Enviamos el prompt completo al LLM y obtenemos su respuesta
    respuesta_llm = llm.invoke(prompt_final)
    # Devolvemos solo el contenido de texto de la respuesta
    return respuesta_llm.content

# --- INTERFAZ DE USUARIO (CHAT) ---
# Mostramos todos los mensajes que ya están en el historial
for msg in st.session_state.chat_history:
    # Determinamos si el mensaje es del asistente o del usuario para mostrarlo correctamente
    tipo = "assistant" if isinstance(msg, AIMessage) else "user"
    with st.chat_message(tipo):
        st.markdown(msg.content)

# Capturamos la nueva pregunta del usuario desde el campo de entrada
if user_query := st.chat_input("Escribe aquí..."):
    # Agregamos la pregunta del usuario al historial y la mostramos en el chat
    st.session_state.chat_history.append(HumanMessage(content=user_query))
    with st.chat_message("user"):
        st.markdown(user_query)
    
    # Generamos la respuesta del asistente y la mostramos
    with st.chat_message("assistant"):
        with st.spinner("Consultando documentos..."):
            # Llamamos a nuestra función RAG manual
            respuesta = responder_pregunta(user_query, st.session_state.chat_history)
            st.markdown(respuesta)
    
    # Agregamos la respuesta del asistente al historial para mantener la conversación
    st.session_state.chat_history.append(AIMessage(content=respuesta))

Writing app.py
